In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# colab에서 matplotlib 한글화할때
!apt-get update -qq
!apt-get install fonts-nanum* -qq
import matplotlib as mpl
import matplotlib.font_manager as fm

fe=fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf',name='NanumGothic')
fm.fontManager.ttflist.insert(0,fe)
plt.rcParams.update({'font.size':18,'font.family':'NanumGothic'})

In [ ]:
import xgboost as xgb
# xgb 전용 매트릭스 DMatrix
dtrain=xgb.DMatrix('/content/drive/MyDrive/daejeon_1/agaricus.txt.train.txt')
dtest=xgb.DMatrix('/content/drive/MyDrive/daejeon_1/agaricus.txt.test.txt')
# regulariztion 규제
# 학습률 : eta
# logistic 회귀 분류 모델
param={'max_depth':2,'eta':1,'silent':1,'objective':'binary:logistic'}
num_round=2
# 모델을 리턴
bst=xgb.train(param,dtrain,num_round)

preds=bst.predict(dtest)
print(preds[:5])

In [ ]:
import pandas as pd
import numpy as np
data_url='http://lib.stat.cmu.edu/datasets/boston'
title=['CRIM','ZM','INDUS','CHAS','NOX','RM','AGE','RAD','TAX','PTRATIO','B','LSTAT','medv']
raw_df=pd.read_csv(data_url,sep='\s+',skiprows=22,header=None)
data=np.hstack([raw_df.values[::2,:],raw_df.values[1::2,:2]])
target=raw_df.values[1::2,2]
print(type(data))
boston_df=pd.DataFrame(data,columns=title)
boston_df.head()

In [ ]:
X=boston_df.iloc[:,:12] # 독립변수
y=boston_df.iloc[:,12]  # 종속변수

In [ ]:
data_dmatrix=xgb.DMatrix(data=X,label=y)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=123)

In [ ]:
from sklearn.metrics import mean_squared_error
# Boost bias를 줄임 : 차를 줄임
# regressor 모델 구성
# 전체 열 중에 30%만 사용
# 학습률 : 경사하강법 : 미분을 구해서 기울기의 방향에 따라 +(기울기가 -), -(기울기 +)
# 최적점을 찾아감: 기울기를 그대로 적용하면 진폭이 커짐
# gamma(L0), alpha(L1), lambda(L2)
xg_reg=xgb.XGBRegressor(booster='gbtree',objective='reg:squarederror',colsample_bytree=0.3,
                        learning_rate=0.1,max_depth=5,alpha=10,n_estimators=10)
xg_reg.fit(X_train,y_train)
preds=xg_reg.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test,preds))
print('RMSE: %f' % (rmse)) # 비교점

In [ ]:
!apt-get -qq install -y graphviz && pip install -q pydot
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
# import matplotlib.pyplot as plt
# xgb.plot_tree(xg_reg,num_trees=0)
# plt.rcParams['figure.figsize']=[50,30]
# plt.show()

In [ ]:
data_dmatrix=xgb.DMatrix(data=X,label=y)
params={'objective':'reg:squarederror','colsample_bytree':0.3,'learning_rate':0.01,
        'max_depth':5,'alpha':10}
# 50회에 걸쳐서 boosting 오차를 개선 
# 끝까지 개선되고 있음
# cv : parameter test하는 명령

cv_results=xgb.cv(dtrain=data_dmatrix,params=params,nfold=3,
                  num_boost_round=50,early_stopping_rounds=10,metrics='rmse',
                  as_pandas=True,seed=123)
print(cv_results.head(30))

In [ ]:
xg_reg=xgb.train(params=params,dtrain=data_dmatrix,num_boost_round=10)

In [ ]:
print((cv_results['test-rmse-mean']).tail(1))
print((cv_results).tail(1))

In [ ]:
# xgb.plot_tree(xg_reg,num_trees=0)
# plt.rcParams['figure.figsize']=[5,5]
# plt.show()

In [ ]:
from xgboost.sklearn import XGBRegressor
import scipy.stats as st
# cpu core 병렬처리
xgbreg=XGBRegressor(objective='reg:squarederror',n_jobs=-1)

In [ ]:
params={'n_estimators':st.randint(3,40),
       'max_depth':st.randint(3,40),
       'learning_rate':st.uniform(0.05,0.4),
       'gamma':st.uniform(0,10)}
# GridSearchCV
from sklearn.model_selection import RandomizedSearchCV # 전체 조합을 사용하지 않고
gs=RandomizedSearchCV(xgbreg,params,n_jobs=1)
gs.fit(X_train,y_train)
gs.best_estimator_

In [ ]:
gs.best_params_

In [ ]:
# pima.csv
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
dataset=pd.read_csv('/content/drive/MyDrive/daejeon_1/pima.csv')
print(dataset.shape)

In [ ]:
dataset.head()

In [ ]:
# X=dataset[dataset.columns[1:]]
X=dataset.iloc[:,1:dataset.shape[1]-1]
print(X.shape)
print(X.head())
y=dataset.iloc[:,dataset.shape[1]-1]
y=np.where(y=='Yes',1,0)
y=pd.Series(y)

In [ ]:
from sklearn.model_selection import train_test_split
seed=7
test_size=0.33
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=test_size,random_state=seed)
# 분류문제
model=XGBClassifier(n_jobs=-1,learning_rate=0.1,max_depth=2,random_state=200)
model.fit(X_train,y_train)
print(model)

In [ ]:
# 변수 중요도 출력함수
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(model)
pyplot.show()

In [ ]:
# 문제 : accuracy_score를 이용해 훈련 정확도 출력
# 훈련데이터에 대한 정확도, 테스트에 정확도 출력
from sklearn.metrics import accuracy_score

y_pred_train=model.predict(X_train)
print(y_pred_train)
accuracy=accuracy_score(y_train,y_pred_train)
print('훈련정확도:%.2f%%'%(accuracy*100))

y_pred=model.predict(X_test)
print(y_pred)
accuracy=accuracy_score(y_test,y_pred)
print('테스트 정확도:%.2f%%'%(accuracy*100))

In [ ]:
# 원인을 noise를 가진 변수라고 보고 feature_selection
from sklearn.feature_selection import SelectFromModel
print(model.feature_importances_)
thresholds=np.sort(model.feature_importances_)
print(thresholds)
print(np.sum(thresholds))

In [ ]:
for thresh in thresholds: # 변수 제거법: threshold 경계선을 중심
  # prefit : 처음부터 다시 학습하지 말고 이전것에다 추가로 학습
  # scikits : transformation(데이터 변형), model(predict)
  selection = SelectFromModel(model, threshold=thresh,prefit=True)
  select_X_train=selection.transform(X_train)
  print(type(select_X_train))
  selection_model=XGBClassifier(use_label_encoder=False)
  selection_model.fit(select_X_train,y_train)
  select_X_test=selection.transform(X_test)
  y_pred=selection_model.predict(select_X_test) # 테스트 데이터 기준으로
  predictions=[round(value) for value in y_pred]
  accuracy=accuracy_score(y_test,predictions)
  print('Thresh=%.3f, n=%d, Accuracy:%.2f%%' %(thresh,
                select_X_train.shape[1],accuracy*100))

In [ ]:
thresh=0.108
selection=SelectFromModel(model,threshold=thresh,prefit=True)
select_X_train=selection.transform(X_train)
select_X_test=selection.transform(X_test)
selection_model=XGBClassifier(use_label_encoder=False)
selection_model.fit(select_X_train,y_train)
select_X_test=selection.transform(X_test)
y_pred=selection_model.predict(select_X_test) # 테스트 데이터 기준으로
predictions=[round(value) for value in y_pred]
accuracy=accuracy_score(y_test,predictions)
print('Thresh=%.3f, n=%d, Accuracy:%.2f%%' %(thresh,
              select_X_train.shape[1],accuracy*100))

In [ ]:
# 문제 
# 위에서 만들어진 모델에 대하여 
# n_estimators, max_depth, learning_rate, gamma에 대해 hyper parameter tunning을
# 실시하고 best_params_을 실시하시오 

params={'n_estimators':st.randint(3,40),
       'max_depth':st.randint(3,40),
       'learning_rate':st.uniform(0.05,0.4),
       'gamma':st.uniform(0,10)}

gs=RandomizedSearchCV(selection_model,params,n_jobs=1,verbose=0)
gs.fit(select_X_train,y_train)
gs.best_params_

In [ ]:
y_pred_train=gs.best_estimator_.predict(select_X_train)
accuracy=accuracy_score(y_train,y_pred_train)
print('훈련정확도: %.2f%%'%(accuracy*100))

In [ ]:
import pickle
pickle.dump(selection_model,open('pima.pickle.dat','wb'))
loaded_model=pickle.load(open('pima.pickle.dat','rb'))

import joblib
joblib.dump(selection_model,'pima.joblib.dat')
loaded_model=joblib.load('pima.joblib.dat')

y_pred=loaded_model.predict(select_X_test)
predictions=[round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)

In [ ]:
# 기존의 모델에 추가 학습이 가능
# fit 할 때 xgb_model 옵션에 = loaded_model
# scikits learn에서는 partial_fit 가 있었음

In [ ]:
append_model=XGBClassifier(use_label_encoder=False)
append_model.fit(select_X_train,y_train,xgb_model=loaded_model)

In [ ]:
y_pred=append_model.predict(select_X_test)
predictions=[round(value) for value in y_pred]

accuracy=accuracy_score(y_test,predictions)
print('Accuracy: %.2f%%'%(accuracy*100))

In [28]:
import pandas as pd
train_set=pd.read_csv('/content/drive/MyDrive/daejeon_1/people.data',header=None)
test_set=pd.read_csv('/content/drive/MyDrive/daejeon_1/people.test',skiprows=1,header=None)

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

In [29]:
train_set.columns=col_labels
test_set.columns=col_labels
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  wage_class      32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [30]:
import numpy as np
# 데이터에 강건한 모델: 트리모델
print(train_set.shape)
np.sum(train_set.isna())
np.sum(train_set.replace(' ?',np.nan).isna())

# 결측치 제거 하기


(32561, 15)


age                  0
workclass         1836
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     583
wage_class           0
dtype: int64

In [31]:
train_nomissing=train_set.replace(' ?',np.nan).dropna()
test_nomissing=test_set.replace(' ?',np.nan).dropna()

In [32]:
test_nomissing['wage_class'].unique()

array([' <=50K.', ' >50K.'], dtype=object)

In [33]:
test_nomissing['wage_class']=test_nomissing.wage_class.replace({' <=50K.':' <=50K',' >50K.':' >50K'})

test_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

In [34]:
combined_set=pd.concat([train_nomissing,test_nomissing],axis=0)
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   fnlwgt          45222 non-null  int64 
 3   education       45222 non-null  object
 4   education_num   45222 non-null  int64 
 5   marital_status  45222 non-null  object
 6   occupation      45222 non-null  object
 7   relationship    45222 non-null  object
 8   race            45222 non-null  object
 9   sex             45222 non-null  object
 10  capital_gain    45222 non-null  int64 
 11  capital_loss    45222 non-null  int64 
 12  hours_per_week  45222 non-null  int64 
 13  native_country  45222 non-null  object
 14  wage_class      45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [35]:
# 문제 : 데이터타입이 object인 열에 대해 중복되지 않는 카테고리가 몇개인지 확인해 보시오

for feature in combined_set.columns:
  if combined_set[feature].dtype =='object':
    print(combined_set[feature].unique())

[' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' Self-emp-inc' ' Without-pay']
[' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' 7th-8th' ' Doctorate' ' Assoc-voc' ' Prof-school'
 ' 5th-6th' ' 10th' ' Preschool' ' 12th' ' 1st-4th']
[' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
[' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Transport-moving' ' Farming-fishing'
 ' Machine-op-inspct' ' Tech-support' ' Craft-repair' ' Protective-serv'
 ' Armed-Forces' ' Priv-house-serv']
[' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
[' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
[' Male' ' Female']
[' United-States' ' Cuba' ' Jamaica' ' India' ' Mexico' ' Puerto-Rico'
 ' Honduras' ' England' ' Canada' ' Germany' ' Iran' ' Philippines'
 ' Poland' ' Colu

In [36]:
# 범주형 데이터 타입인 열을 모두 범주화 하시오 

for feature in combined_set.columns:
  if combined_set[feature].dtype =='object':
    combined_set[feature]=pd.Categorical(combined_set[feature]).codes
    print(combined_set.head())

   age  workclass  fnlwgt   education  education_num       marital_status  \
0   39          5   77516   Bachelors             13        Never-married   
1   50          4   83311   Bachelors             13   Married-civ-spouse   
2   38          2  215646     HS-grad              9             Divorced   
3   53          2  234721        11th              7   Married-civ-spouse   
4   28          2  338409   Bachelors             13   Married-civ-spouse   

           occupation    relationship    race      sex  capital_gain  \
0        Adm-clerical   Not-in-family   White     Male          2174   
1     Exec-managerial         Husband   White     Male             0   
2   Handlers-cleaners   Not-in-family   White     Male             0   
3   Handlers-cleaners         Husband   Black     Male             0   
4      Prof-specialty            Wife   Black   Female             0   

   capital_loss  hours_per_week  native_country wage_class  
0             0              40   United-St

In [37]:
# final_train/final_test 로 분리
combined_set.head()
print(train_nomissing.shape[0])
print(test_nomissing.shape)

final_train=combined_set[:train_nomissing.shape[0]]
final_test=combined_set[train_nomissing.shape[0]:]

30162
(15060, 15)


In [38]:
# 문제
# 독립변수/ 종속변수(wage_class) 분리하기
# y_train, y_test

y_train=final_train.pop('wage_class')
y_test=final_test.pop('wage_class')


In [39]:
from xgboost import XGBClassifier
xgb1=XGBClassifier(tree_method='gpu_hist',gpu_id=0)
xgb1.fit(final_train,y_train)
print(xgb1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)


In [40]:
result=xgb1.predict(final_test)
print(result)
score=(result==y_test).mean()
print(score)

[0 0 0 ... 1 0 1]
0.8672642762284196


In [41]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
cv_params={'max_depth':[3,5,7],'min_child_weight':[1,3,5],
           'n_estimators':[100,300,500,1000]}
ind_params={'learning_rate':0.1,'seed':0,'subsample':0.8,
            'colsample_bytree':0.8,'objective':'binary:logistic'}
optimized_GBM=GridSearchCV(xgb.XGBClassifier(**ind_params),
                           cv_params,scoring='accuracy',cv=5,n_jobs=-1)
optimized_GBM.fit(final_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, ma...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [100, 300, 500, 1000]},
             scoring='accuracy')

In [42]:
estimator_best=optimized_GBM.best_estimator_
estimator_best.save_model('model.json')

# 웹서버에서
estimator_best_loaded=XGBClassifier(tree_method='gpu_hist',gpu_id=0)
estimator_best_loaded.load_model('model.json')
estimator_best_loaded

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['int', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', 'int', 'int', 'int', 'int', 'int'],
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [43]:
import xgboost
# 추가학습
# XGBClassifier로 만든 estimator_bset_loaded
booster_best=estimator_best_loaded.get_booster()
updatetrain=xgboost.DMatrix(final_train,y_train) # 무변화 ( 데이터가 변겨이 되면 가능)
booster_best.update(updatetrain,1)